# 🚀 ProjetoX - Automação de Vídeos YouTube com IA

Sistema completo para gerar vídeos profissionais automaticamente!

## 🎯 Nicho Escolhido: Conteúdo Religioso
- Histórias Bíblicas Animadas
- Mensagens de Fé
- Curiosidades Bíblicas

## 🎬 O que este notebook faz:
1. Gera roteiro com ChatGPT
2. Cria personagens cartoon 3D
3. Gera narração profissional
4. Anima as cenas
5. Aplica lip-sync
6. Edita vídeo final

⏱️ Tempo total: 15-30 minutos por vídeo  
💰 Custo estimado: R$13-23 por vídeo

---

## ⚠️ ANTES DE COMEÇAR:

1. Configure suas API keys nos **Secrets do Colab** (ícone 🔑 na barra lateral)
2. Tenha pelo menos $10 USD em créditos nas APIs
3. Leia o README.md para detalhes

Vamos começar! 🔥

In [ ]:
# === CÉLULA 2: Instalação de Dependências ===
print("📦 Instalando dependências...")

!pip install -q openai==1.12.0
!pip install -q elevenlabs==0.2.27
!pip install -q replicate==0.22.0
!pip install -q moviepy==1.0.3
!pip install -q pydub==0.25.1
!pip install -q Pillow==10.2.0
!pip install -q tqdm==4.66.1

# Instalar ffmpeg (necessário para MoviePy)
!apt-get -qq install -y ffmpeg

print("✅ Dependências instaladas!")

In [ ]:
# === CÉLULA 3: Clone do Repositório ===
import os

# Clonar repositório se não existir
if not os.path.exists('/content/projetoX'):
    print("📥 Clonando repositório ProjetoX...")
    !git clone https://github.com/Ganzer-Publicidade/projetoX.git /content/projetoX
    print("✅ Repositório clonado!")
else:
    print("✅ Repositório já existe!")

# Adicionar ao path
import sys
sys.path.append('/content/projetoX')

In [ ]:
# === CÉLULA 4: Montar Google Drive ===
from google.colab import drive

print("💾 Montando Google Drive...")
drive.mount('/content/drive')

# Criar diretório de output no Drive
output_dir = '/content/drive/MyDrive/ProjetoX_Videos'
os.makedirs(output_dir, exist_ok=True)

print(f"✅ Drive montado! Vídeos serão salvos em: {output_dir}")

In [ ]:
# === CÉLULA 5: Carregar API Keys (SEGURO) ===
from google.colab import userdata

print("🔑 Carregando API Keys dos Secrets do Colab...")

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ELEVENLABS_API_KEY = userdata.get('ELEVENLABS_API_KEY')
    REPLICATE_API_TOKEN = userdata.get('REPLICATE_API_TOKEN')
    
    # Validar que não estão vazias
    if not all([OPENAI_API_KEY, ELEVENLABS_API_KEY, REPLICATE_API_TOKEN]):
        raise ValueError("Uma ou mais API keys estão vazias!")
    
    print("✅ API Keys carregadas com sucesso!")
    print(f"   - OpenAI: {OPENAI_API_KEY[:8]}...")
    print(f"   - ElevenLabs: {ELEVENLABS_API_KEY[:8]}...")
    print(f"   - Replicate: {REPLICATE_API_TOKEN[:8]}...")
    
except Exception as e:
    print("❌ ERRO ao carregar API Keys!")
    print(f"   {e}")
    print("\n📚 COMO CONFIGURAR:")
    print("   1. Clique no ícone 🔑 na barra lateral esquerda")
    print("   2. Clique em 'Add new secret'")
    print("   3. Adicione cada chave:")
    print("      - Nome: OPENAI_API_KEY")
    print("      - Nome: ELEVENLABS_API_KEY")
    print("      - Nome: REPLICATE_API_TOKEN")
    print("   4. Toggle: Enable notebook access")
    print("   5. Execute esta célula novamente")
    raise

In [ ]:
# === CÉLULA 6: Configuração do Vídeo ===
# @title 🎬 Configuração do Vídeo

# Nicho do vídeo
NICHO = "conteudo_religioso"  # @param ["conteudo_religioso", "historias_infantis", "terror", "curiosidades"]

# Tema do vídeo
TEMA = "A História de Davi e Golias - A Vitória da Fé"  # @param {type:"string"}

# Duração desejada (minutos)
DURACAO_MINUTOS = 5  # @param {type:"slider", min:3, max:15, step:1}

# Idioma
IDIOMA = "pt-br"  # @param ["pt-br", "en", "es"]

# Duração de cada cena (segundos) - ECONOMIA!
DURACAO_CENA_SEGUNDOS = 5  # @param {type:"slider", min:3, max:10, step:1}

# Usar GPT-3.5 (mais barato) ou GPT-4?
USAR_GPT35 = True  # @param {type:"boolean"}

print("=" * 60)
print("🎬 CONFIGURAÇÃO DO VÍDEO")
print("=" * 60)
print(f"📝 Tema: {TEMA}")
print(f"🏷️ Nicho: {NICHO}")
print(f"⏱️ Duração: {DURACAO_MINUTOS} minutos")
print(f"🌍 Idioma: {IDIOMA}")
print(f"🎞️ Duração por cena: {DURACAO_CENA_SEGUNDOS}s")
print(f"🤖 Modelo: {'GPT-3.5-turbo (econômico)' if USAR_GPT35 else 'GPT-4 (premium)'}")
print("=" * 60)

# Estimativa de custo
custo_roteiro = 0.02 if USAR_GPT35 else 0.10
custo_personagens = 0.20
custo_audio = 0.60
custo_animacao = (DURACAO_MINUTOS * 60 // DURACAO_CENA_SEGUNDOS) * 0.06
custo_lipsync = 0.50
custo_total = custo_roteiro + custo_personagens + custo_audio + custo_animacao + custo_lipsync

print(f"\n💰 Custo Estimado:")
print(f"   - Roteiro: ${custo_roteiro:.2f}")
print(f"   - Personagens: ${custo_personagens:.2f}")
print(f"   - Áudio: ${custo_audio:.2f}")
print(f"   - Animação: ${custo_animacao:.2f}")
print(f"   - Lip-sync: ${custo_lipsync:.2f}")
print(f"   🔥 TOTAL: ${custo_total:.2f} (≈R${custo_total * 5:.2f})")

In [ ]:
# === CÉLULA 7: Executar Pipeline ===
from src.pipeline import VideoAutomationPipeline
from datetime import datetime

# Configuração completa
config = {
    'nicho': NICHO,
    'tema': TEMA,
    'duracao_minutos': DURACAO_MINUTOS,
    'idioma': IDIOMA,
    'api_keys': {
        'openai': OPENAI_API_KEY,
        'elevenlabs': ELEVENLABS_API_KEY,
        'replicate': REPLICATE_API_TOKEN
    },
    'output_dir': output_dir,
    'optimization': {
        'cena_duration_seconds': DURACAO_CENA_SEGUNDOS,
        'use_gpt35': USAR_GPT35,
        'enable_cache': True
    }
}

print("\n" + "=" * 70)
print("🚀 INICIANDO GERAÇÃO DO VÍDEO")
print("=" * 70)
print(f"⏰ Início: {datetime.now().strftime('%H:%M:%S')}")
print("\nIsso pode levar 15-30 minutos...")
print("☕ Aproveite para tomar um café!\n")

# Criar e executar pipeline
try:
    pipeline = VideoAutomationPipeline(config)
    video_path = pipeline.executar_completo()
    
    print("\n" + "=" * 70)
    print("🎉 VÍDEO GERADO COM SUCESSO!")
    print("=" * 70)
    print(f"📁 Localização: {video_path}")
    print(f"⏰ Fim: {datetime.now().strftime('%H:%M:%S')}")
    print("\n✅ Vídeo salvo no seu Google Drive!")
    
except Exception as e:
    print("\n" + "=" * 70)
    print("❌ ERRO NA GERAÇÃO DO VÍDEO")
    print("=" * 70)
    print(f"Erro: {e}")
    print("\n📚 Consulte a documentação ou abra uma issue no GitHub")
    raise

In [ ]:
# === CÉLULA 8: Visualizar Vídeo ===
from IPython.display import Video

if 'video_path' in locals() and os.path.exists(video_path):
    print("👀 PREVIEW DO VÍDEO:")
    display(Video(video_path, width=800))
else:
    print("⚠️ Vídeo não encontrado. Execute a célula anterior primeiro.")

In [ ]:
# === CÉLULA 9: Download do Vídeo ===
from google.colab import files

if 'video_path' in locals() and os.path.exists(video_path):
    print("⬇️ Baixando vídeo...")
    files.download(video_path)
    print("✅ Download iniciado!")
else:
    print("⚠️ Vídeo não encontrado.")

In [ ]:
# === CÉLULA 10: Estatísticas ===
if 'pipeline' in locals():
    print("📊 ESTATÍSTICAS:")
    
    # Calcular tempo total
    if hasattr(pipeline, 'tempo_total_segundos'):
        print(f"   ⏱️ Tempo total: {pipeline.tempo_total_segundos}s")
    
    # Custo real
    if hasattr(pipeline, 'custo_total'):
        print(f"   💰 Custo real: ${pipeline.custo_total:.2f}")
    
    # Estatísticas do roteiro
    if hasattr(pipeline, 'roteiro') and pipeline.roteiro:
        num_cenas = len(pipeline.roteiro.get('cenas', []))
        print(f"   🎬 Cenas geradas: {num_cenas}")
    
    # Estatísticas de áudio
    if hasattr(pipeline, 'audios'):
        print(f"   🎙️ Áudios gerados: {len(pipeline.audios)}")
    
    # Duração final
    if hasattr(pipeline, 'duracao_final'):
        print(f"   📏 Duração final: {pipeline.duracao_final}s")
else:
    print("⚠️ Execute o pipeline primeiro para ver estatísticas.")

# 🎉 Parabéns! Seu vídeo está pronto!

## 📤 Próximos Passos:

### 1. Upload no YouTube
- Vá para: https://studio.youtube.com
- Clique em "Criar" > "Enviar vídeos"
- Faça upload do vídeo
- Adicione título otimizado para SEO
- Escreva descrição detalhada
- Adicione tags relevantes
- Escolha thumbnail atrativa

### 2. Gerar Mais Vídeos
- Volte para a Célula 6
- Mude o TEMA
- Execute novamente!

### 3. Escalar Produção
- Crie vídeos em inglês e espanhol
- Automatize com Google Apps Script
- Crie playlists temáticas

### 4. Monetização
- Ative monetização (1K inscritos + 4K horas)
- Links de afiliados na descrição
- Venda produtos digitais

---

## 💡 Ideias de Vídeos:

**Histórias Bíblicas:**
- A História de Moisés
- Noé e a Arca
- José do Egito
- Daniel na Cova dos Leões
- A Ressurreição de Jesus

**Mensagens:**
- Versículos para Ansiedade
- Salmo 23 Explicado
- Como Ter Fé nos Momentos Difíceis

---

## 📚 Documentação Completa:
Leia o README.md no repositório para mais detalhes!

**BORA DOMINAR O YOUTUBE! 🚀🔥**